In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import random
import pandas as pd
import numpy as np
import os
import csv

from torchvision import transforms

from tqdm.auto import tqdm

from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

D:\conda\envs\quality\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
CFG = {
    'EPOCHS':100,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':32,
    'SEED':42
}

In [26]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Funtion Define

In [28]:
def RemoveEmptyColumn(train_X, test):
    train_X = train_X.dropna(axis=1, how='all')
    test = test[train_X.columns]

    return train_X, test

def DropDuplicateColumns(train_X, test):
    train_X = train_X.loc[:,~train_X.T.duplicated(keep='first')]
    test = test[train_X.columns]

    return train_X, test

def RemoveOneValueColumn(train_X, test):
    for col in [x for x in train_X.columns if 'X_' in x]:
        if len(train_X[col].value_counts())==1:
            train_X = train_X.drop(col, axis=1)
        
    test = test[train_X.columns]

    return train_X, test

def ConcatProdLine(train_X, test):
    train_X['PROD_LINE'] = train_X['PRODUCT_CODE']+'_'+train_X['LINE']
    train_X = train_X.drop(['PRODUCT_CODE','LINE'],axis=1)
    test['PROD_LINE'] = test['PRODUCT_CODE']+'_'+test['LINE']
    test = test.drop(['PRODUCT_CODE','LINE'],axis=1)

    return train_X, test

def fillNa(train_X, test):
    train_X = train_X.fillna(0)
    test = test.fillna(0)
    
    return train_X, test

def DatascalingRobust(train_X, test):
    scaler = RobustScaler()

    num_features_train = [x for x in train_X.columns if "X" in x]
    #train_x.select_dtypes(exclude=['object']).columns.to_list()

    train_X[num_features_train] = scaler.fit_transform(train_X[num_features_train])
    test[num_features_train] = scaler.transform(test[num_features_train])

    return train_X, test

def OnehotEncoder(train_X, test):
    dummies_col = []
    for c in ['PROD_LINE']:
        df = pd.get_dummies(train_X[c])
        train_X[df.columns] = df
        train_X = train_X.drop(c, axis=1)
        df = pd.get_dummies(test[c])
        test[df.columns] = df
        test = test.drop(c, axis=1)
        dummies_col.extend(df.columns)
        
    return train_X, test, dummies_col

# Data Load

In [29]:
def load_data(df, mode):
    train_X = train_df.drop(columns=['Y_Quality', 'Y_Class', 'TIMESTAMP', 'PRODUCT_ID'])
    if mode == 'single':
        train_y = train_df['Y_Class']
    if mode == 'dual':
        train_y = train_df[['Y_Class', 'Y_Quality']]
        
    return train_X, train_y

In [30]:
train_df = pd.read_csv('../open/train.csv')
test_df = pd.read_csv('../open/test.csv')

In [31]:
train_X, train_y = load_data(train_df, 'dual')
test = test_df.drop(columns=['TIMESTAMP', 'PRODUCT_ID'])

train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_df['Y_Class'])

In [32]:
def GetMask(train_X):
    mask_dict = {}
    for prod in train_X['PRODUCT_CODE'].unique():
        mask_dict[prod]={}
    for prod in train_X['PRODUCT_CODE'].unique():
        for line in train_X['LINE'].unique():
            cols=train_X[(train_X['PRODUCT_CODE']==prod)&(train_X['LINE']==line)].dropna(axis=1, how='all').columns
            mask_dict[prod][line]=[True]*len(train_X.columns)
            for col in cols:
                mask_dict[prod][line][list(train_X.columns).index(col)]=False
    return mask_dict

# Data Preprocessing

In [33]:
# 우선은 test 대신에 val_X 로 대체 
train_X, val_X = RemoveEmptyColumn(train_X, val_X)
train_X, val_X = DropDuplicateColumns(train_X, val_X)
train_X, val_X = RemoveOneValueColumn(train_X, val_X)
mask_dict = GetMask(train_X)
#train_X, val_X = ConcatProdLine(train_X, val_X)
train_X, val_X = fillNa(train_X, val_X)
train_X, val_X = DatascalingRobust(train_X, val_X)
#train_X, val_X, prod_dum = OnehotEncoder(train_X, val_X)

위 데이터 전처리시 train_X 와 val_X 의 number of columns 가 다름. PROD_LINE 에서 val_X 가 하나 부족.

# CustomDataset

In [34]:
class CustomDataset(Dataset):
    def __init__(self, train_X, train_y, mode='train'):
        super(CustomDataset, self).__init__()
        self.train_X = train_X.values
        if mode=='train':
            self.train_y = train_y.values
        elif mode=='test':
            self.train_y = None
        
    def __len__(self):
        return len(self.train_X)
    
    def __getitem__(self, idx):
        line, prod = self.train_X[idx][:2]
        mask = mask_dict[prod][line][2:]
        x = np.array(self.train_X[idx][2:], dtype=np.float64)
        
        if mode=='train':
            y = np.array(self.train_y[idx], dtype=np.float64)
            return torch.tensor(x), torch.tensor(y), torch.tensor(mask)
        elif mode=='test':
            return return torch.tensor(x), torch.tensor(y), torch.tensor(mask)

In [35]:
train_dataset = CustomDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, batch_size = 8, shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_X, val_y)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle=False, num_workers=0)

# Model Define

In [36]:
class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(1601, 8)
        self.Classification = nn.Linear(8, 1)
        self.Regression = nn.Linear(8, 1)

    def forward(self, x, mask):
        x.masked_fill_(mask, -1000.)
        x = self.hidden(x)
        y1= self.Classification(x)
        y2= self.Regression(x)
        return y1, y2

# Train

In [37]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    best_val_acc = - np.inf   # init to negative infinity
    model.to(device)
    loss_fn1 = nn.CrossEntropyLoss().to(device) # 다중클래스분류 손실 함수
    loss_fn2 = nn.MSELoss().to(device) # 회귀 손실 함수
    
    model.train()
    
    for epoch in range(1, CFG['EPOCHS']):
        train_loss = []
        for x, y, mask in tqdm(iter(train_loader)):
            x = x.float().to(device)
            y1 = y[:,0].float().to(device)
            y2 = y[:,1].float().to(device)
            mask = mask.to(device)
            
            optimizer.zero_grad()   # clear gradients 
            pre_class, pre_regression = model(x, mask)
            
            loss = loss_fn1(y1, pre_class.squeeze(dim=-1))
            loss2 = loss_fn2(y2, pre_regression.squeeze(dim=-1))
            loss_total = loss + loss2
            loss_total.backward()
            
            optimizer.step()
            
            train_loss.append(loss_total.item())
            
        _val_loss, _val_acc = validation(model, loss_fn1, loss_fn2, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
        
    return best_model

# Validation

In [38]:
def validation(model, loss_fn1, loss_fn2, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    for x, y, mask in tqdm(iter(train_loader)):
        x = x.float().to(device)
        y1 = y[:,0].float().to(device)
        y2 = y[:,1].float().to(device)
        mask = mask.to(device)
        
        pre_class, pre_regression = model(x, mask)
        
        loss = loss_fn1(y1, pre_class.squeeze(dim=-1))
        loss2 = loss_fn2(y2, pre_regression.squeeze(dim=-1))
        loss_total = loss + loss2
        
        pre_class  = pre_class.cpu().detach().numpy()
        y1 = y1.cpu().detach().numpy()
        
        preds = pre_class > 0.5
        
        batch_acc = (y1 == preds).mean()
        
        val_acc.append(batch_acc)
        val_loss.append(loss.item())
        
    _val_loss = np.mean(val_loss)
    _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

In [39]:
model = Multiclass()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-8, verbose=True)

best_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 164.79it/s]


Epoch [1], Train Loss : [67162439.64928] Val Loss : [-61148.76663] Val ACC : [0.14792]


100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 139.58it/s]


Epoch [2], Train Loss : [20718529.71497] Val Loss : [-89397.67461] Val ACC : [0.14792]


100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 156.97it/s]


...

Epoch [97], Train Loss : [-175700.50781] Val Loss : [-203211.33418] Val ACC : [0.16097]


100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 161.32it/s]


Epoch [98], Train Loss : [-176900.70150] Val Loss : [-200683.10872] Val ACC : [0.15929]


100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 161.97it/s]

Epoch [99], Train Loss : [-174020.66289] Val Loss : [-200304.35143] Val ACC : [0.15929]


In [ ]:
def predict(model, dataset, device):
    model.eval()
    tqdm_dataset = tqdm(enumerate(dataset))
    results = []
    for x, _, mask in tqdm(iter(train_loader)):
        x = x.float().to(device)
        mask = mask.to(device)

        optimizer.zero_grad()   # clear gradients 
        pre_class, pre_regression = model(x, mask)
        with torch.no_grad():
            output = model(img, seq)
        output = torch.tensor(torch.argmax(output, dim=1), dtype=torch.int32).cpu().numpy()
        results.extend(output)
    return results

model = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model.load_state_dict(torch.load(save_path, map_location=device))
model.to(device)

preds = predict(test_dataloader)

In [ ]:
val_dataset = CustomDataset(val_X, val_y)
